# Interaction with RChain network via Python/Notebooks

## 1. setup

In current docker compose configuration we have the node name in the environments

In [ ]:
RNODE_HOST = %env NODE_NAME

Open a channel with `grpc` using our Python library.

In [ ]:
from rchain_grpc import casper

connection = casper.create_connection(host=RNODE_HOST)
connection

## 2. simple deploy

Write code in RhoLang

In [ ]:
rholang_code = """
new print(`rho:io:stdout`) in {
    print!("Hello World!")
}
"""

You can deploy/propose to the node you're connected with the string containing Rholang code

In [ ]:
casper.deploy(connection, rholang_code)

In [ ]:
casper.propose(connection)

## 3. advanced deploy

A more interesting example is to run a contract and listen to the node for an answer on the deployed block.

To make this trick we define a placeholder name that will be used in the final deployed contract to listen and show the content of block we propose.

In [ ]:
output_placeholder = "your_output_placeholder"

rholang_code = """
%s!("bar")
""" % output_placeholder

In [ ]:
output = casper.run_and_get_value_from(
    connection=connection,
    term=rholang_code, 
    output_placeholder=output_placeholder
)

In [ ]:
# verify the status
if output.get('status').lower() == 'success':
    # all is good, pretty print results
    print("Results:")
    from pprint import pprint
    pprint(output.get('blockResults'))
else:
    # uh-oh something went wrong
    print("Failed!")